In [1]:
# @title Clique no play ▶️ para executar essa célula e instalar as bibiliotecas necessárias { run: "auto", vertical-output: true, display-mode: "form" }
%%capture
!pip install elevenlabs
!pip install pydub
!pip install nltk

In [2]:
# @title Execute essa célula para gerar a sua interface { form-width: "50%" }



from elevenlabs import generate
from pydub import AudioSegment
import io
import string
import ipywidgets as widgets
from IPython.display import display, Audio, HTML, clear_output

# Função para dividir a sentença em partes
def split_sentence(sentence, max_length):
    sentences = []
    current_chunk = ""

    for char in sentence:
        current_chunk += char
        if len(current_chunk) >= max_length and char in string.punctuation:
            sentences.append(current_chunk.strip())
            current_chunk = ""

    if current_chunk:
        sentences.append(current_chunk.strip())

    # Se não houver pontuações dentro do limite, dividir na próxima pontuação até 247 caracteres
    if len(sentences[-1]) < max_length and len(sentence) > max_length:
        remaining_chunk = sentence[len(sentences[-1]):]
        for char in remaining_chunk:
            current_chunk += char
            if len(current_chunk) >= max_length and char in string.punctuation:
                sentences.append(current_chunk.strip())
                current_chunk = ""

    return sentences

# Função para gerar áudio a partir de uma sentença
def generate_audio(sentence):
    audio = generate(
        text=sentence,
        voice="Thomas",
        model='eleven_multilingual_v1'
    )
    audio_data = io.BytesIO(audio)
    audio_segment = AudioSegment.from_mp3(audio_data)
    return audio_segment

# Função para juntar áudios em ordem
def concatenate_audios(audios):
    combined_audio = AudioSegment.empty()
    for audio_segment in audios:
        combined_audio += audio_segment
    return combined_audio

# Função para gerar e reproduzir áudios
def generate_and_play_audio(b):
    # Limpa a saída anterior
    clear_output(wait=True)

    # Atualiza a mensagem de status
    status_label.value = "Gerando seu áudio! Aguarde alguns segundos..."

    # Divide o texto em partes
    sentences = split_sentence(text_input.value, 230)

    # Gera áudios para cada parte
    audios = [generate_audio(sentence) for sentence in sentences]

    # Junta todos os áudios em um único áudio
    combined_audio = concatenate_audios(audios)

    # Exporta e reproduz o áudio
    combined_audio.export("combined_audio.mp3", format="mp3")
    display(Audio(filename="combined_audio.mp3", autoplay=True))

    # Atualiza a mensagem de status
    status_label.value = "Áudio gerado com sucesso!"

# Adiciona frases na interface com estilo de fundo preto
info_text = widgets.HTML(
    value="<font color='cadetblue' size='+1'>"
          "<i>Desenvolvido por Gabriel Carvalho, para Synthetics</i><br><br>"
          "Cole seu texto na caixa abaixo (máx 2 mil caracteres) e depois clique em <b>Gerar Áudio</b>:<br><br>"
          "Depois de gerar esse áudio, vá até a setinha ao lado de <i>RAM e Disco</i> e depois clique em <i>Desassociar e eliminar tempo de execução</i> e depois em Sim."
          "</font>",
    layout=widgets.Layout(width='80%')
)
display(info_text)

# Adiciona a caixa de texto
text_input = widgets.Textarea(
    value='',
    placeholder='Cole seu texto aqui...',
    layout=widgets.Layout(width='80%', height='200px')
)
display(text_input)

# Adiciona o botão de gerar áudio
generate_button = widgets.Button(description='Gerar Áudio')
generate_button.on_click(generate_and_play_audio)
display(generate_button)

# Adiciona um rótulo para exibir a mensagem de status
status_label = widgets.Label(value="")
display(status_label)


HTML(value="<font color='cadetblue' size='+1'><i>Desenvolvido por Gabriel Carvalho, para Synthetics</i><br><br…

Textarea(value='', layout=Layout(height='200px', width='80%'), placeholder='Cole seu texto aqui...')

Button(description='Gerar Áudio', style=ButtonStyle())

Label(value='')